#first we integerate the data and label using panda an os libraries

In [ ]:
import os
import pandas as pd
import torch.nn as nn
import torch.optim as optim


dataset_path = "/Users/nadamourad/Desktop/PatternRecognition/Husky-Wolf-Binary-Classifier/dataset"
categories = ["huskies", "wolves"]
data = []
for category in categories:
    folder_path = os.path.join(dataset_path, category) 
    label = categories.index(category)
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        data.append([image_path, label])  # Store path and label
images = pd.DataFrame(data, columns=["image_path", "label"])
print(images.head)


<bound method NDFrame.head of                                            image_path  label
0   /Users/nadamourad/Desktop/PatternRecognition/H...      0
1   /Users/nadamourad/Desktop/PatternRecognition/H...      0
2   /Users/nadamourad/Desktop/PatternRecognition/H...      0
3   /Users/nadamourad/Desktop/PatternRecognition/H...      0
4   /Users/nadamourad/Desktop/PatternRecognition/H...      0
..                                                ...    ...
95  /Users/nadamourad/Desktop/PatternRecognition/H...      1
96  /Users/nadamourad/Desktop/PatternRecognition/H...      1
97  /Users/nadamourad/Desktop/PatternRecognition/H...      1
98  /Users/nadamourad/Desktop/PatternRecognition/H...      1
99  /Users/nadamourad/Desktop/PatternRecognition/H...      1

[100 rows x 2 columns]>


In [2]:
from sklearn.model_selection import train_test_split

train_im, test_im = train_test_split(images, test_size=0.2, random_state=42, stratify=images["label"])
print(train_im.head())


                                           image_path  label
15  /Users/nadamourad/Desktop/PatternRecognition/H...      0
52  /Users/nadamourad/Desktop/PatternRecognition/H...      1
60  /Users/nadamourad/Desktop/PatternRecognition/H...      1
66  /Users/nadamourad/Desktop/PatternRecognition/H...      1
68  /Users/nadamourad/Desktop/PatternRecognition/H...      1


now we need to preprocess the dataset

In [3]:
import torch  
from torch.utils.data import Dataset, DataLoader  
from torchvision import transforms
import cv2 
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define augmentation transformations for training data
train_transforms = A.Compose([
    A.Resize(128, 128),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=20, p=0.5),
    A.RandomResizedCrop(size=(128, 128), scale=(0.8, 1.0), ratio=(0.75, 1.33), p=0.5),
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ToTensorV2(),
    ])
test_transforms = A.Compose([
    A.Resize(128, 128),
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ToTensorV2(),
])



creating dataset class

In [4]:
class HuskyWolfDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe  
        self.transform = transform 
    def __len__(self):
        return len(self.dataframe)  
    def __getitem__(self, index):
        image_path = self.dataframe.iloc[index]["image_path"]  
        label = self.dataframe.iloc[index]["label"]  

        # Load image using OpenCV
        image = cv2.imread(image_path)  # Read image (BGR format)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Apply transformations (if any)
        if self.transform:
            augmented = self.transform(image=image)  # Apply transformations
            image = augmented["image"]

        return image, torch.tensor(label, dtype=torch.long)  # Return image & label as tensors


making instance of it

In [5]:
train_dataset = HuskyWolfDataset(train_im, transform=train_transforms)  
test_dataset = HuskyWolfDataset(test_im, transform=test_transforms)  


# Create DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)


In [9]:


class MLPClassifier(nn.Module):
    def __init__(self, input_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)  # First hidden layer
        self.fc2 = nn.Linear(512, 256)  # Second hidden layer
        self.fc3 = nn.Linear(256, 128)  # Third hidden layer
        self.fc4 = nn.Linear(128, 1)  # Output layer (1 neuron for binary classification)
        self.relu = nn.ReLU()  # ReLU activation function
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation for probability output

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x



In [10]:
input_size = 128 * 128 * 3  
model = MLPClassifier(input_size)

# Define Loss Function and Optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.view(images.size(0), -1)  # Flatten images
        labels = labels.float().unsqueeze(1)  # Convert labels to float

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/10], Loss: 0.6667
Epoch [2/10], Loss: 0.9809
Epoch [3/10], Loss: 0.2756
Epoch [4/10], Loss: 0.2058
Epoch [5/10], Loss: 0.2493
Epoch [6/10], Loss: 0.0695
Epoch [7/10], Loss: 0.0554
Epoch [8/10], Loss: 0.1815
Epoch [9/10], Loss: 0.1011
Epoch [10/10], Loss: 0.0549
